In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2
import geopy
import json
import geojson

In [2]:
#!pip install geojson

In [3]:
file = pd.read_excel('resources/income/allhlcn18.xlsx')
file.head()

,Area Code,St,Cnty,Own,NAICS,Year,Qtr,Area Type,St Name,Area,Ownership,Industry,Annual Average Status Code,Annual Average Establishment Count,Annual Average Employment,Annual Total Wages,Annual Average Weekly Wage,Annual Average Pay,Employment Location Quotient Relative to U.S.,Total Wage Location Quotient Relative to U.S.
0,US000,US,0.0,0,10,2018,A,Nation,NaN,U.S. TOTAL,Total Covered,"10 Total, all industries",NaN,10011038,146131754,8368407405945,1101,57266,1.0,1.0
1,US000,US,0.0,1,10,2018,A,Nation,NaN,U.S. TOTAL,Federal Government,"10 Total, all industries",NaN,58935,2795195,233837729742,1609,83657,1.0,1.0
2,US000,US,0.0,2,10,2018,A,Nation,NaN,U.S. TOTAL,State Government,"10 Total, all industries",NaN,69945,4624977,280971551393,1168,60751,1.0,1.0
3,US000,US,0.0,3,10,2018,A,Nation,NaN,U.S. TOTAL,Local Government,"10 Total, all industries",NaN,170478,14159744,729438687540,991,51515,1.0,1.0
4,US000,US,0.0,5,10,2018,A,Nation,NaN,U.S. TOTAL,Private,"10 Total, all industries",NaN,9711681,124551838,7124159437270,1100,57198,1.0,1.0


In [4]:
df = file[["Area\nCode","St","Area Type","Area", "St Name", "Ownership"]]
df.head()

,Area Code,St,Area Type,Area,St Name,Ownership
0,US000,US,Nation,U.S. TOTAL,NaN,Total Covered
1,US000,US,Nation,U.S. TOTAL,NaN,Federal Government
2,US000,US,Nation,U.S. TOTAL,NaN,State Government
3,US000,US,Nation,U.S. TOTAL,NaN,Local Government
4,US000,US,Nation,U.S. TOTAL,NaN,Private


In [5]:
states_only = df[(df["Ownership"]=="Total Covered") & 
                 (df["Area Type"].str.contains("Nation|State"))].fillna("United States of America")
states_only.head()


,Area Code,St,Area Type,Area,St Name,Ownership
0,US000,US,Nation,U.S. TOTAL,United States of America,Total Covered
18,01000,01,State,Alabama -- Statewide,Alabama,Total Covered
1191,02000,02,State,Alaska -- Statewide,Alaska,Total Covered
1722,04000,04,State,Arizona -- Statewide,Arizona,Total Covered
2026,05000,05,State,Arkansas -- Statewide,Arkansas,Total Covered


In [6]:
states_info_df = states_only.rename(columns = {"Area\nCode":"FIPS", 
                               "St Name": "State Name"})
states_info_df = states_info_df[["State Name","FIPS", "St", "Area Type"]].reset_index(drop=True)

abbrv = ["US", "AL", "AK", "AZ", "AR", "CA","CO","CT","DE","DC","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN", "MS", "MO", "MT", "NE", "NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
states_info_df['Abbrv'] = abbrv
states_info_df.head()

,State Name,FIPS,St,Area Type,Abbrv
0,United States of America,US000,US,Nation,US
1,Alabama,01000,01,State,AL
2,Alaska,02000,02,State,AK
3,Arizona,04000,04,State,AZ
4,Arkansas,05000,05,State,AR


In [7]:
states_info_df.to_csv('resources/State_Info.csv', index=False)

In [8]:
state_shapes = gpd.read_file('us-states.geo.json')
state_shapes.head()

,id,name,density,geometry
0,01,Alabama,94.650,"POLYGON ((-87.359 35.001, -85.607 34.985, -85...."
1,02,Alaska,1.264,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2..."
2,04,Arizona,57.050,"POLYGON ((-109.043 37.000, -109.048 31.332, -1..."
3,05,Arkansas,56.430,"POLYGON ((-94.474 36.502, -90.153 36.496, -90...."
4,06,California,241.700,"POLYGON ((-123.233 42.006, -122.379 42.012, -1..."


In [9]:
state_shapes = state_shapes.rename(columns = {"name":"State Name"}).drop(columns=["density"])
state_shapes.head()

,id,State Name,geometry
0,01,Alabama,"POLYGON ((-87.359 35.001, -85.607 34.985, -85...."
1,02,Alaska,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2..."
2,04,Arizona,"POLYGON ((-109.043 37.000, -109.048 31.332, -1..."
3,05,Arkansas,"POLYGON ((-94.474 36.502, -90.153 36.496, -90...."
4,06,California,"POLYGON ((-123.233 42.006, -122.379 42.012, -1..."


In [10]:
us_info_gpd = pd.merge(state_shapes, states_info_df, how='outer', on='State Name')
us_info_gpd.tail()

,id,State Name,geometry,FIPS,St,Area Type,Abbrv
48,54,West Virginia,"POLYGON ((-80.51860 40.63695, -80.51860 39.722...",54000,54,State,WV
49,55,Wisconsin,"POLYGON ((-90.41543 46.56848, -90.22921 46.508...",55000,55,State,WI
50,56,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",56000,56,State,WY
51,72,Puerto Rico,"POLYGON ((-66.44834 17.98433, -66.77148 18.006...",NaN,NaN,NaN,NaN
52,NaN,United States of America,None,US000,US,Nation,US


In [11]:
states_info_gpd = us_info_gpd.iloc[0:51]
states_info_gpd.head()

,id,State Name,geometry,FIPS,St,Area Type,Abbrv
0,01,Alabama,"POLYGON ((-87.359 35.001, -85.607 34.985, -85....",01000,01,State,AL
1,02,Alaska,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2...",02000,02,State,AK
2,04,Arizona,"POLYGON ((-109.043 37.000, -109.048 31.332, -1...",04000,04,State,AZ
3,05,Arkansas,"POLYGON ((-94.474 36.502, -90.153 36.496, -90....",05000,05,State,AR
4,06,California,"POLYGON ((-123.233 42.006, -122.379 42.012, -1...",06000,06,State,CA


In [12]:
states_info_poly = pd.DataFrame(states_info_gpd)
states_info_poly
states_info_poly.to_excel('state_info_polygons.xlsx')

In [16]:
case_file = pd.read_csv('new_hiv_cases.csv')
case_file = case_file.rename(columns={"Name":"State Name"})
case_file.head()

,State Name,New HIV Case Rate 2008-2012,New HIV Case Rate 2013-2018,New HIV Case Rate Variance
0,Alabama,14.385351,13.219613,1.165738
1,Alaska,4.147444,3.996431,0.151014
2,Arizona,9.750057,10.556607,-0.806551
3,Arkansas,8.503143,9.644722,-1.141579
4,California,14.223003,12.548216,1.674786


In [17]:
states_cases_gpd = pd.merge(states_info_gpd, case_file, how='outer', on='State Name')
states_cases_gpd.head()

,id,State Name,geometry,FIPS,St,Area Type,Abbrv,New HIV Case Rate 2008-2012,New HIV Case Rate 2013-2018,New HIV Case Rate Variance
0,01,Alabama,"POLYGON ((-87.359 35.001, -85.607 34.985, -85....",01000,01,State,AL,14.385351,13.219613,1.165738
1,02,Alaska,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2...",02000,02,State,AK,4.147444,3.996431,0.151014
2,04,Arizona,"POLYGON ((-109.043 37.000, -109.048 31.332, -1...",04000,04,State,AZ,9.750057,10.556607,-0.806551
3,05,Arkansas,"POLYGON ((-94.474 36.502, -90.153 36.496, -90....",05000,05,State,AR,8.503143,9.644722,-1.141579
4,06,California,"POLYGON ((-123.233 42.006, -122.379 42.012, -1...",06000,06,State,CA,14.223003,12.548216,1.674786


In [18]:
states_cases_gpd = states_cases_gpd.rename(columns={"New HIV Case Rate 2008-2012":"Before Prep", "New HIV Case Rate 2013-2018":"After Prep", "New HIV Case Rate Variance":"Variance"})
states_cases_gpd.head()

,id,State Name,geometry,FIPS,St,Area Type,Abbrv,Before Prep,After Prep,Variance
0,01,Alabama,"POLYGON ((-87.359 35.001, -85.607 34.985, -85....",01000,01,State,AL,14.385351,13.219613,1.165738
1,02,Alaska,"MULTIPOLYGON (((-131.602 55.118, -131.569 55.2...",02000,02,State,AK,4.147444,3.996431,0.151014
2,04,Arizona,"POLYGON ((-109.043 37.000, -109.048 31.332, -1...",04000,04,State,AZ,9.750057,10.556607,-0.806551
3,05,Arkansas,"POLYGON ((-94.474 36.502, -90.153 36.496, -90....",05000,05,State,AR,8.503143,9.644722,-1.141579
4,06,California,"POLYGON ((-123.233 42.006, -122.379 42.012, -1...",06000,06,State,CA,14.223003,12.548216,1.674786


In [19]:
states_cases_gpd.to_file('new_HIV_case_rate.geojson', driver='GeoJSON')